In [1]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime

In [2]:
### 3901.csv in Dataframe (EGID)

egid = pd.read_csv('EGID/3901/3901.csv', delimiter=';')

# Spalte URL hinzufügen
egid['URL'] = 'https://www.housing-stat.ch/regbl/resources/public/geb_public/' + egid['EGID'].astype('str') + '?lang=de'


In [3]:
# URL herunterladen, als pdf abspeichern

# alle URLS in Liste, für download in Schlaufe
url_liste = egid.values.tolist()

urls = []
url_egid = {}

for url_list in url_liste:
    # Dictionary je Zeile aus Dataframe
    einzelne_egid = url_list[0]
    einzelne_url = url_list[13]
    
    url_egid = {'EGID': einzelne_egid, 'URL': einzelne_url}
    urls.append(url_egid)

# jede URL runterladen und als pdf abspeichern
for url in tqdm(urls):
    
    inhalt = requests.get(url['URL'], allow_redirects=True)
    time.sleep(5)
    with open('PDF/GEB_PUBLIC' + str(url['EGID']) + '.pdf', 'wb') as pdffile:
        pdffile.write(inhalt.content)
        
        

100%|██████████| 5107/5107 [8:14:40<00:00,  5.81s/it]   


In [17]:
# PDF auslesen
# EGID, Gebäudenummer, Koordinaten etc.

import os
import textract
from tqdm import tqdm
import pandas as pd


# leere Liste für alle Gebäude
alle_gebaeude = []

# PDF in Liste
pdfs = [f for f in os.listdir('PDF/') if f.endswith('.pdf') or f.endswith('.PDF')]

for pdf in tqdm(pdfs):

    text = textract.process('PDF/' + pdf, method='pdfminer', encoding='ISO-8859-1')
    text = text.decode("ISO-8859-1").replace("\n", "xxxxxxxxxx").split("xxxxxxxxxx")


    
    #minidict grad mit den gewünschten elementen
    
    startposition = text.index('ParzNr.') + 3 # manchmal mehr Adressen oben, daher absolute Startposition ParzNr. + 3
    
    
    gebaeude = {'Grundstueck-Nummer': int(text[startposition].replace(' ','')),
               'EGID': int(text[startposition + 31].replace(' ','')),
               'Gebaeude-Koordinate X': int(text[startposition + 5].replace(' ','')),
               'Gebaeude-Koordinate Y': int(text[startposition + 9].replace(' ','')),
               'Gebaeude-Baujahr': int(text[startposition + 15].replace(' ','')),
               'Gebaeude-Kategorie': text[startposition + 42] + " " + text[startposition + 43],
                'Gebaeude-Klasse-Nummer': text[startposition + 37],
               'Gebaeude-Klasse': text[startposition + 44] + " " + text[startposition + 45],
               'Gebaeude-Flaeche': int(text[startposition + 46].replace(' ','')),
               'Gebaeude-Geschosse': int(text[startposition + 51].replace(' ','')),
               'Gebaeude-Strasse': text[startposition + 68],
               'Gebaeude-Hausnummer': text[startposition + 71],
               'Gebaeude-PLZ': text[startposition + 74],
               'Gebaeude-Ort': text[startposition + 77]}
    
    alle_gebaeude.append(gebaeude)
 


# csv abspeichern
file = pd.DataFrame(alle_gebaeude)            
headernames = ['Grundstueck-Nummer', 'EGID', 'Gebaeude-Koordinate X', 'Gebaeude-Koordinate Y',
               'Gebaeude-Baujahr', 'Gebaeude-Kategorie', 'Gebaeude-Klasse-Nummer',
               'Gebaeude-Klasse', 'Gebaeude-Flaeche', 'Gebaeude-Geschosse', 'Gebaeude-Strasse',
               'Gebaeude-Hausnummer', 'Gebaeude-PLZ', 'Gebaeude-Ort']

file.to_csv('Gebäude_Chur.csv', header=headernames, sep=";", index=False)


  0%|          | 0/5011 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 10: 'Gebäudekoordinaten'

In [15]:
# TEST

# PDF auslesen
# EGID, Gebäudenummer, Koordinaten etc.

import os
import textract
from tqdm import tqdm
import pandas as pd


# leere Liste für alle Gebäude
alle_gebaeude = []

# PDF in Liste
pdfs = [f for f in os.listdir('PDF/') if f.endswith('.pdf') or f.endswith('.PDF')]

for pdf in tqdm(pdfs):

    pdf = 'GEB_PUBLIC190821530.PDF'
    text = textract.process('PDF/' + pdf, method='pdfminer', encoding='ISO-8859-1')
    text = text.decode("ISO-8859-1").replace("\n", "xxxxxxxxxx").split("xxxxxxxxxx")


    
    #minidict grad mit den gewünschten elementen
    
    startposition = text.index('ParzNr.') + 3 # manchmal mehr Adressen oben, daher absolute Startposition ParzNr. + 3
    
    
    gebaeude = {'Grundstueck-Nummer': int(text[startposition].replace(' ','')),
               'EGID': int(text[startposition + 31].replace(' ','')),
               'Gebaeude-Koordinate X': int(text[startposition + 5].replace(' ','')),
               'Gebaeude-Koordinate Y': int(text[startposition + 9].replace(' ','')),
               'Gebaeude-Baujahr': int(text[startposition + 15].replace(' ','')),
               'Gebaeude-Kategorie': text[startposition + 42] + " " + text[startposition + 43],
                'Gebaeude-Klasse-Nummer': text[startposition + 37],
               'Gebaeude-Klasse': text[startposition + 44] + " " + text[startposition + 45],
               'Gebaeude-Flaeche': int(text[startposition + 46].replace(' ','')),
               'Gebaeude-Geschosse': int(text[startposition + 51].replace(' ','')),
               'Gebaeude-Strasse': text[startposition + 68],
               'Gebaeude-Hausnummer': text[startposition + 71],
               'Gebaeude-PLZ': text[startposition + 74],
               'Gebaeude-Ort': text[startposition + 77]}
    
    alle_gebaeude.append(gebaeude)
    
 
    break

# csv abspeichern
file = pd.DataFrame(alle_gebaeude)            
headernames = ['Grundstueck-Nummer', 'EGID', 'Gebaeude-Koordinate X', 'Gebaeude-Koordinate Y',
               'Gebaeude-Baujahr', 'Gebaeude-Kategorie', 'Gebaeude-Klasse-Nummer',
               'Gebaeude-Klasse', 'Gebaeude-Flaeche', 'Gebaeude-Geschosse', 'Gebaeude-Strasse',
               'Gebaeude-Hausnummer', 'Gebaeude-PLZ', 'Gebaeude-Ort']

file.to_csv('Gebäude_Chur.csv', header=headernames, sep=";", index=False)


  0%|          | 0/5011 [00:02<?, ?it/s]


In [16]:
gebaeude

{'Grundstueck-Nummer': 5992,
 'EGID': 190821530,
 'Gebaeude-Koordinate X': 2758072,
 'Gebaeude-Koordinate Y': 1190874,
 'Gebaeude-Baujahr': 2013,
 'Gebaeude-Kategorie': 'Mehrfamilienhaus, ohne  Nebennutzung',
 'Gebaeude-Klasse-Nummer': '1122',
 'Gebaeude-Klasse': 'Gebäude mit drei oder mehr  Wohnungen',
 'Gebaeude-Flaeche': 5565,
 'Gebaeude-Geschosse': 6,
 'Gebaeude-Strasse': 'Rheinfelsstrasse',
 'Gebaeude-Hausnummer': '21',
 'Gebaeude-PLZ': '7000',
 'Gebaeude-Ort': 'Chur'}